In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from IPython.display import FileLink
from imutils import paths
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.random.seed(0)

import random
import shutil
import cv2
import os

In [3]:
import pickle

In [21]:
from PIL import Image
import numpy as np
from skimage import transform

In [170]:
RFC_Model = pickle.load(open('RF_model.pkl', 'rb'))
LR_Model = pickle.load(open('LR_model.pkl', 'rb'))
SVM_Model = pickle.load(open('SVM_model.pkl', 'rb'))

In [16]:
import keras
from keras.applications import VGG16
from keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator


In [17]:
reconstructed_model = keras.models.load_model("keras_model")

In [205]:
def predict_single(img_file):
    heading = ['model', 'probability', 'prediction']
    covid_RF = predict_Covid_RF(img_file)
    covid_LR = predict_Covid_LR(img_file)
    covid_SVM = predict_Covid_RF(img_file)
    pneu = pneumonia_CNN(img_file)
    out = np.vstack((heading, covid_RF, covid_LR, covid_SVM, pneu))
    return (out)


In [201]:
RF_list = []
def predict_Covid_RF(img_file):
    'function to take image and return prediction'
    test_image = cv2.imread(img_file)
    test_image = cv2.cvtColor(test_image, cv2.IMREAD_GRAYSCALE)
    test_image = cv2.resize(test_image, (224, 224))
    test_img = test_image.flatten().reshape(1, -1)
    

    RFC_pred_prob = RFC_Model.predict_proba(test_img)
    RFC_pred = RFC_Model.predict(test_img)

    RF_list = ['RF_Covid', RFC_pred_prob[0,0], RFC_pred[0]]

    return (RF_list)


In [202]:
LR_list = []
def predict_Covid_LR(img_file):
    'function to take image and return prediction'
    test_image = cv2.imread(img_file)
    test_image = cv2.cvtColor(test_image, cv2.IMREAD_GRAYSCALE)
    test_image = cv2.resize(test_image, (224, 224))
    test_img = test_image.flatten().reshape(1, -1)
    

    LR_pred_prob = LR_Model.predict_proba(test_img)
    LR_pred = LR_Model.predict(test_img)

    LR_list = ['LR_Covid', LR_pred_prob[0,0], LR_pred[0]]

    return (LR_list)


In [203]:
SVM_list = []
def predict_Covid_SVM(img_file):
    'function to take image and return prediction'
    test_image = cv2.imread(img_file)
    test_image = cv2.cvtColor(test_image, cv2.IMREAD_GRAYSCALE)
    test_image = cv2.resize(test_image, (224, 224))
    test_img = test_image.flatten().reshape(1, -1)
    

    SVM_pred_prob = 'Na'
    SVM_pred = SVM_Model.predict(test_img)

    SVM_list = ['SVM_Covid', SVM_pred_prob, SVM_pred[0]]

    return (SVM_list)


In [180]:
    predict_Covid_SVM('dataset/covid/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg')

['SVM_Covid', 'Na', 'covid']

In [41]:
arrayList = []

In [92]:
def iload(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (150, 150, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image


In [157]:
def pneumonia_CNN(img_file): 
    iload(img_file)
    image = iload(img_file)
    pred_pnemonia = (1-reconstructed_model.predict(image))
    if reconstructed_model.predict(image) <0.5: 
        out_pneu= 'normal'
    if reconstructed_model.predict(image) >0.5: 
        out_pneu = 'pneumonia'
    arrayList = ['Pneumonia_CNN', pred_pnemonia[0,0], out_pneu]
    return(arrayList)

In [206]:
predict_single('archive/chest_xray/test/NORMAL/IM-0001-0001.jpeg')

array([['model', 'probability', 'prediction'],
       ['RF_Covid', '0.93', 'covid'],
       ['LR_Covid', '0.9999991825824696', 'covid'],
       ['RF_Covid', '0.93', 'covid'],
       ['Pneumonia_CNN', '0.99973696', 'normal']], dtype='<U18')

In [155]:
pneumonia_CNN('archive/chest_xray/train/NORMAL/IM-0115-0001.jpeg')

['Pneumonia_CNN', 'normal', 0.9999961]

### How we buit it 
We sourced an opensource chest x-ray data set with labels for normal and pneuamonia, and a subset of these indivduals had labels for normal and covid (n=, n= . 